In [226]:
#Script pour exécuter des commandes shell et créer des csv à partir de requêtes SQL dans un conteneur Docker SQLite
import subprocess

def run_command(command, shell=False):
    """Helper to run shell or list-based commands with error handling."""
    try:
        result = subprocess.run(command, shell=shell, check=True, capture_output=True, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {command}")
        print(e.stderr)



In [227]:
#Start the container using the shell script
print("🚀 Starting Docker container with start_sqlite.sh...")
COMPOSE_FILE="../docker/docker-compose-sqlite.yml"
run_command(["docker", "compose", "-f", COMPOSE_FILE, "up", "-d", "--build"])


🚀 Starting Docker container with start_sqlite.sh...
#1 [internal] load local bake definitions
#1 reading from stdin 572B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile.sqlite
#2 transferring dockerfile: 185B done
#2 DONE 0.0s

#3 [auth] library/alpine:pull token for registry-1.docker.io
#3 DONE 0.0s

#4 [internal] load metadata for docker.io/library/alpine:3.19
#4 DONE 0.9s

#5 [internal] load .dockerignore
#5 transferring context: 277B done
#5 DONE 0.0s

#6 [1/3] FROM docker.io/library/alpine:3.19@sha256:6baf43584bcb78f2e5847d1de515f23499913ac9f12bdf834811a3145eb11ca1
#6 resolve docker.io/library/alpine:3.19@sha256:6baf43584bcb78f2e5847d1de515f23499913ac9f12bdf834811a3145eb11ca1 done
#6 DONE 0.0s

#7 [2/3] RUN apk add --no-cache sqlite bash
#7 CACHED

#8 [3/3] WORKDIR /data
#8 CACHED

#9 exporting to image
#9 exporting layers done
#9 exporting manifest sha256:c62068895fb5c5a190e6f8ddea60b4068e0cf398f6cbcf7d6a453e973dd3226b done
#9 exporting config sha256:670c78

In [228]:
# #Copy the SQLite DB into the container
# print("📁 Copying newIMDB.db into docker container...")
# run_command(["docker", "cp", "../data/db/newIMDB.db", "moviesdb_sqlite:/data/newIMDB.db"])



In [229]:
#copy the SQL file into the container
print("📁 Copying marvel_movies.sql into docker container...")
run_command(["docker", "cp", "../utils/marvel_movies.sql", "moviesdb_sqlite:/data/marvel_movies.sql"])

📁 Copying marvel_movies.sql into docker container...



In [230]:
#Run the SQL file using sqlite3 inside the container
print("📄 Executing marvel_queries.sql inside the container...")
run_command([
    "docker", "exec", "-i", "moviesdb_sqlite",
    "sqlite3", "/data/newIMDB.db", ".read /data/marvel_movies.sql"
])

📄 Executing marvel_queries.sql inside the container...



In [ ]:
# import sqlite3

# # Connect to your SQLite database
# conn = sqlite3.connect('../data/db/newIMDB.db')  # Adjust path as needed
# cursor = conn.cursor()

# # Your SQL query
# query = """

# SELECT 
#     title_id AS tconst,
#     primary_title AS primaryTitle,
#     premiered AS startYear,
#     runtime_minutes AS runtimeMinutes,
#     genres
# FROM titles
# WHERE 
#     (type IN ('movie', 'tv_series'))
#     AND premiered >= 2008 
#     AND (
#         primary_title LIKE '%Iron Man%'
#         OR primary_title LIKE '%Captain America%'
#         OR primary_title LIKE 'Thor:%'
#         OR primary_title LIKE '%The Avengers%'
#         OR primary_title LIKE 'Avengers:%'
#         OR primary_title LIKE '%Guardians of the galaxy%'
#         OR primary_title LIKE '%Ant-Man%'
#         OR primary_title LIKE '% Hulk %'
#         OR primary_title LIKE 'Black Panther'
#         OR primary_title LIKE 'Black Panther:'
#         OR primary_title LIKE '%Doctor Strange%'
#         OR primary_title LIKE '%Captain Marvel%'
#         OR primary_title LIKE '%Spider-Man%'
#         OR primary_title LIKE '%Black Widow%'
#         OR primary_title LIKE 'Shang-Chi%'
#         OR primary_title LIKE '%WandaVision%'
#         OR primary_title LIKE 'Loki'
#         OR primary_title LIKE '%Hawkeye%'
#         OR primary_title LIKE '%Moon Knight%'
#         OR primary_title LIKE '%Ms. Marvel%'
#         OR primary_title LIKE '%She-Hulk%'
#         OR primary_title LIKE '%Thor: Ragnarok%'
#         OR primary_title LIKE 'Eternals'
#         OR primary_title LIKE 'Deadpool%'
#         OR primary_title LIKE 'The Marvels'
#     )
#     AND primary_title NOT LIKE '%viking%'
#     AND primary_title NOT LIKE '%lego%'
#     AND runtime_minutes IS NOT NULL
#     AND (
#         genres LIKE '%Action%' 
#         OR genres LIKE '%Adventure%' 
#         OR genres LIKE '%Sci-Fi%'
#     )
#     AND genres NOT LIKE '%Family%'
#     AND genres NOT LIKE '%Animation%'
# ORDER BY premiered, title_id;
# """

# # Execute and fetch
# cursor.execute(query)
# row = cursor.fetchmany()

# # Check if something was returned
# if row:
#     print("✅ Query returned at least one result:", row)
# else:
#     print("❌ Query returned nothing.")

# # Clean up
# cursor.close()
# conn.close()

In [231]:
#copy csv file created by marvel_queries.sql from container to local
run_command([
    "docker", "cp", "moviesdb_sqlite:/data/marvel_movies.csv", "../data/tests/marvel_movies.csv"
])

In [232]:
#read csv into pandas dataframe
import pandas as pd
marvel_movies_df = pd.read_csv("../data/tests/marvel_movies.csv")
print("Returned rows:", len(marvel_movies_df))
marvel_movies_df.head()

Returned rows: 71


,tconst,primaryTitle,startYear,runtimeMinutes,genres
0,tt0371746,Iron Man,2008,126,"Action,Adventure,Sci-Fi"
1,tt4664384,Shamelessly She-Hulk,2009,90,"Action,Comedy,Drama"
2,tt1228705,Iron Man 2,2010,124,"Action,Sci-Fi"
3,tt0458339,Captain America: The First Avenger,2011,124,"Action,Adventure,Sci-Fi"
4,tt0848228,The Avengers,2012,143,"Action,Sci-Fi"


In [256]:
marvel_movies_df.to_csv("../data/tests/marvel_movies_cleaned.csv", index=False)


In [233]:
#copy the SQL file into the container
print("📁 Copying marvel_characters.sql into docker container...")
run_command(["docker", "cp", "../utils/marvel_characters.sql", "moviesdb_sqlite:/data/marvel_characters.sql"])

📁 Copying marvel_characters.sql into docker container...



In [234]:
#Run the SQL file using sqlite3 inside the container
print("📄 Executing marvel_characters.sql inside the container...")

run_command([
    "docker", "exec", "-i", "moviesdb_sqlite",
    "sh", "-c", "sqlite3 /data/newIMDB.db < /data/marvel_characters.sql"
])  

📄 Executing marvel_characters.sql inside the container...



In [235]:
run_command([
    "docker", "cp", "moviesdb_sqlite:/data/marvel_characters.csv", "../data/tests/marvel_characters.csv"
])

In [ ]:
# import sqlite3

# # Connect to your SQLite database
# conn = sqlite3.connect('../data/db/newIMDB.db')  # Adjust path as needed
# cursor = conn.cursor()

# # Your SQL query
# query = """

# WITH
#     marvel_titles AS (
#         SELECT title_id
#         FROM titles
# WHERE 
#     (type IN ('movie', 'tv_series'))
#     AND premiered >= 2008 
#     AND (
#         primary_title LIKE '%Iron Man%'
#         OR primary_title LIKE '%Captain America%'
#         OR primary_title LIKE 'Thor:%'
#         OR primary_title LIKE '%The Avengers%'
#         OR primary_title LIKE 'Avengers:%'
#         OR primary_title LIKE '%Guardians of the galaxy%'
#         OR primary_title LIKE '%Ant-Man%'
#         OR primary_title LIKE '% Hulk %'
#         OR primary_title LIKE 'Black Panther'
#         OR primary_title LIKE 'Black Panther:'
#         OR primary_title LIKE '%Doctor Strange%'
#         OR primary_title LIKE '%Captain Marvel%'
#         OR primary_title LIKE '%Spider-Man%'
#         OR primary_title LIKE '%Black Widow%'
#         OR primary_title LIKE 'Shang-Chi%'
#         OR primary_title LIKE '%WandaVision%'
#         OR primary_title LIKE 'Loki'
#         OR primary_title LIKE '%Hawkeye%'
#         OR primary_title LIKE '%Moon Knight%'
#         OR primary_title LIKE '%Ms. Marvel%'
#         OR primary_title LIKE '%She-Hulk%'
#         OR primary_title LIKE '%Thor: Ragnarok%'
#         OR primary_title LIKE 'Eternals'
#         OR primary_title LIKE 'Deadpool%'
#         OR primary_title LIKE 'The Marvels'
#     )
#     AND primary_title NOT LIKE '%viking%'
#     AND primary_title NOT LIKE '%lego%'
#     AND runtime_minutes IS NOT NULL
#     AND (
#         genres LIKE '%Action%' 
#         OR genres LIKE '%Adventure%' 
#         OR genres LIKE '%Sci-Fi%'
#     )
#     AND genres NOT LIKE '%Family%'
#     AND genres NOT LIKE '%Animation%'
#     )
# SELECT
#     titles.title_id AS title_id,
#     characters AS character_name
# FROM titles
#     JOIN crew ON titles.title_id = crew.title_id
# WHERE
#     titles.title_id IN (
#         SELECT title_id
#         FROM marvel_titles
#     )
#     AND characters IS NOT NULL
#     AND characters NOT LIKE '%Self%'
#     AND LOWER(category) != 'self'
#     AND category NOT LIKE 'archive_%'
# ORDER BY character_name ASC;
# """

# # Execute and fetch
# cursor.execute(query)
# row = cursor.fetchmany()

# # Check if something was returned
# if row:
#     print("✅ Query returned at least one result:", row)
# else:
#     print("❌ Query returned nothing.")

# # Clean up
# cursor.close()
# conn.close()



In [254]:
# read csv into pandas dataframe
import pandas as pd
marvel_characters_df = pd.read_csv("../data/tests/marvel_characters.csv")

# marvel_characters_df['character_name'] = marvel_characters_df['character_name'].apply(lambda x: x[0] if isinstance(x,list) and len(x) > 0 else None)
# print("Returned rows:", len(marvel_characters_df))
# character_name is formatted as a string representation of a list, convert it back to list and get first element
import ast


marvel_characters_df['character_name'] = marvel_characters_df['character_name'].str.replace(r"\s*\([^)]*\)", "", regex=True)


# marvel_characters_df['character_name'].apply(
#     lambda x: print(x, len(x)) if x is not None else None
# )
marvel_characters_df['character_name'] = marvel_characters_df['character_name'].apply(
    lambda x: ast.literal_eval(x)[0] if pd.notnull(x) else None
)

marvel_characters_df.head()

,title_id,character_name
0,tt10676048,Aamir Khan
1,tt2250912,Aaron Davis
2,tt0371746,Abu Bakaar
3,tt21053530,Adrian Quinn
4,tt10782740,Adrian Toomes


In [255]:
marvel_characters_df.to_csv("../data/tests/marvel_characters_cleaned.csv", index=False)


In [ ]:
# #Stop and remove the container
# print("🚀 Stopping Docker container with stop_sqlite.sh...")
# COMPOSE_FILE="../docker/docker-compose-sqlite.yml"
# CONTAINER_NAME="moviesdb_sqlite"

# run_command(["docker", "stop", CONTAINER_NAME])
# run_command(["docker", "compose", "-f", COMPOSE_FILE, "down"])

🚀 Stopping Docker container with stop_sqlite.sh...
moviesdb_sqlite


